In [46]:
import pandas  as pd
import numpy as np 
import os  
import sys
#from pattern.en import  singularize

In [47]:
train=pd.read_csv('mailDF.csv')

In [48]:
train.head()

,MailId,MailSubject,MailCorpus
0,1.0,"Hyper-V host maintenance on Monday, 3rd March ...",FYI &
1,2.0,"Hyper-V host maintenance on Monday, 3rd March ...",", Please note the updated list of servers w..."
2,3.0,"Hyper-V host maintenance on Monday, 3rd March ...",", Migrations of VMs completed and all serve..."
3,4.0,"Hyper-V host maintenance on Monday, 3rd March ...",", Issue with QGRLADPDEV migration has been ..."
4,159.0,"Hyper-V host maintenance on Monday, 3rd March ...",", Please note we would be doing some urgent..."


In [49]:
id=train['MailId']
from nltk.stem.porter import *

In [50]:
import sklearn.feature_extraction.text as text
from sklearn.feature_extraction.text import CountVectorizer
import nltk 

In [51]:
from nltk.stem.snowball import SnowballStemmer

In [52]:
stemmer = SnowballStemmer("english")

In [53]:
stemmer

In [54]:
def tokenize_and_stem(text):
   
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    #filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in  tokens]
    return tokens
    

In [55]:
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = "english",   
                             max_features = 3000) 

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
import string

In [57]:
%time dtm = vectorizer.fit_transform(train['MailCorpus']).toarray()

CPU times: user 16 ms, sys: 8 ms, total: 24 ms
Wall time: 31.5 ms


In [58]:
vocab = np.array(vectorizer.get_feature_names())

In [59]:
vocab

array(['00', '00pm', '03', ..., 'yellow', 'yes', 'yusuf'], 
      dtype='<U32')

In [60]:
import lda
num_topics =7
 
model = lda.LDA(n_topics=num_topics,n_iter=3000, random_state=1,refresh=500)

In [61]:
%time doctopic =model.fit_transform(dtm)
np.set_printoptions(suppress=True)  
doc_topic = model.doc_topic_
topic_assigned=[]

INFO:lda:n_documents: 180
INFO:lda:vocab_size: 1023
INFO:lda:n_words: 3933
INFO:lda:n_topics: 7
INFO:lda:n_iter: 3000
INFO:lda:<0> log likelihood: -40764
INFO:lda:<500> log likelihood: -25789
INFO:lda:<1000> log likelihood: -25890
INFO:lda:<1500> log likelihood: -25861
INFO:lda:<2000> log likelihood: -25852
INFO:lda:<2500> log likelihood: -25793
INFO:lda:<2999> log likelihood: -25793


CPU times: user 12.5 s, sys: 0 ns, total: 12.5 s
Wall time: 12.6 s


In [62]:
topic_assigned=[]
 
#add  topic assigned 
for i in range(0,train.shape[0]):
 
    #print("{} (top topic: {})".format(id[i], doc_topic[i].argmax()))
    topic_assigned.append(doc_topic[i].argmax())
train["topic_assigned"]=np.array(topic_assigned)


In [63]:
train.head()

,MailId,MailSubject,MailCorpus,topic_assigned
0,1.0,"Hyper-V host maintenance on Monday, 3rd March ...",FYI &,5
1,2.0,"Hyper-V host maintenance on Monday, 3rd March ...",", Please note the updated list of servers w...",1
2,3.0,"Hyper-V host maintenance on Monday, 3rd March ...",", Migrations of VMs completed and all serve...",4
3,4.0,"Hyper-V host maintenance on Monday, 3rd March ...",", Issue with QGRLADPDEV migration has been ...",4
4,159.0,"Hyper-V host maintenance on Monday, 3rd March ...",", Please note we would be doing some urgent...",1


In [64]:

n_top_words=np.int(12)
topic_word = model.topic_word_
topic_list=[]
train_topic=[]
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    s='Topic {}: {}'.format(i, ' '.join(topic_words))
    #s=[singularize(word) for  word  in  s.split()]
    #s=[word for  word  in  s.split()]
    s=s.join(s)
    
    topic_list.append(s)
for i in range(0,train.shape[0]):
    train_topic.append(topic_list[train['topic_assigned'][i]])
train_topic=np.array(train_topic)
train["topic_content"]=train_topic

In [65]:
train.tail(10)

,MailId,MailSubject,MailCorpus,topic_assigned,topic_content
170,155.0,"Maintenance on QGNTVS01 on Saturday, 21st Marc...",", Maintenance job is complete and all serve...",5,TTopic 5: 00 pm cluster maintenance resources ...
171,147.0,"Hyper-V host maintenance on Saturday, 28th Mar...",", Please note we would be applying security...",6,TTopic 6: team snoc 3088 ext managed qgrladpde...
172,148.0,"Hyper-V host maintenance on Saturday, 28th Mar...",", Please note the modified list as below: ...",6,TTopic 6: team snoc 3088 ext managed qgrladpde...
173,149.0,"Hyper-V host maintenance on Saturday, 28th Mar...",", All the below servers are un-managed fro...",6,TTopic 6: team snoc 3088 ext managed qgrladpde...
174,156.0,"Hyper-V host maintenance on Saturday, 28th Mar...",", As discussed, please remove QGRLBIPRD03 ...",0,TTopic 0: ts server service need activity work...
175,157.0,"Hyper-V host maintenance on Saturday, 28th Mar...",", Just a gentle reminder about patcng activ...",0,TTopic 0: ts server service need activity work...
176,158.0,"Hyper-V host maintenance on Saturday, 28th Mar...",", Updates completed and all servers are bac...",3,TTopic 3: servers updates online completed qgn...
177,150.0,About BWise incident I1503-522 - DWH is not wo...,", If no one replied by 4:00 PM today, plea...",0,TTopic 0: ts server service need activity work...
178,151.0,About BWise incident I1503-522 - DWH is not wo...,", Please restart QGNTSQLDEV\APPDEV @4:00 to...",0,TTopic 0: ts server service need activity work...
179,180.0,About BWise incident I1503-522 - DWH is not wo...,"Mr. El-Mokadem, On 2015-03-11 20:23 you re...",0,TTopic 0: ts server service need activity work...


# 

In [66]:
train['topic_content'][178]

'TTopic 0: ts server service need activity working database kindly restart mail reminder servicesoTopic 0: ts server service need activity working database kindly restart mail reminder servicespTopic 0: ts server service need activity working database kindly restart mail reminder servicesiTopic 0: ts server service need activity working database kindly restart mail reminder servicescTopic 0: ts server service need activity working database kindly restart mail reminder services Topic 0: ts server service need activity working database kindly restart mail reminder services0Topic 0: ts server service need activity working database kindly restart mail reminder services:Topic 0: ts server service need activity working database kindly restart mail reminder services Topic 0: ts server service need activity working database kindly restart mail reminder servicestTopic 0: ts server service need activity working database kindly restart mail reminder servicessTopic 0: ts server service need activi

In [34]:
current_string.join(" ")

' '

In [36]:
current_string.join(['a','b','c'])

'aalexbalexc'